<a href="https://colab.research.google.com/github/Eden-Green12/Dropout-regularization/blob/main/Dropout_regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd 
import numpy as np

In [2]:
url = 'https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/master/13_dropout_layer/sonar_dataset.csv'

In [3]:
df = pd.read_csv(url,header=None)

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [5]:
df.shape

(208, 61)

In [7]:
df[60].value_counts()

M    111
R     97
Name: 60, dtype: int64

In [8]:
X = df.drop(60,axis='columns')
y = df[60]

In [9]:
y.head()

0    R
1    R
2    R
3    R
4    R
Name: 60, dtype: object

In [10]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094


In [11]:
y = pd.get_dummies(y,drop_first = True)

In [12]:
y.head()

,R
0,1
1,1
2,1
3,1
4,1


In [13]:
y.value_counts()

R
0    111
1     97
dtype: int64

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test ,y_train,y_test = train_test_split(X,y,test_size = 0.25, random_state = 1)

In [15]:
X_train.shape

(156, 60)

In [16]:
X_test.shape

(52, 60)

In [17]:
y_train.shape

(156, 1)

In [19]:
import tensorflow as tf 
from tensorflow import keras 

In [20]:
model = keras.Sequential([
                          keras.layers.Dense(60, input_dim = 60 , activation='relu'),
                          keras.layers.Dense(20,activation = 'relu'),
                          keras.layers.Dense(30,activation = 'relu'),
                          keras.layers.Dense(1,activation = 'sigmoid'),

])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

model.fit(X_train,y_train,epochs = 100, batch_size = 8)

Epoch 1/100
20/20 [==============================] - 3s 4ms/step - loss: 0.6791 - accuracy: 0.6218
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6531 - accuracy: 0.6987
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6284 - accuracy: 0.7051
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5933 - accuracy: 0.7372
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5529 - accuracy: 0.7756
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5116 - accuracy: 0.7756
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4663 - accuracy: 0.8269
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4427 - accuracy: 0.8077
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4150 - accuracy: 0.7821
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3811 - accuracy: 0.8654
Epoch 11/

**The model is overfitting** 

In [21]:
model.evaluate(X_test,y_test)

2/2 [==============================] - 0s 8ms/step - loss: 0.7039 - accuracy: 0.8269


[0.7039427161216736, 0.8269230723381042]

In [24]:
y_pred = model.predict(X_test).reshape(-1)
print(y_pred[:10])

y_pred = np.round(y_pred)
print(y_pred[:10])

[4.7306276e-07 4.0122083e-01 9.9273127e-01 1.5772241e-05 9.9999917e-01
 9.9993753e-01 1.3277869e-01 1.0000000e+00 3.7214733e-05 9.9999964e-01]
[0. 0. 1. 0. 1. 1. 0. 1. 0. 1.]


In [25]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.93      0.85        27
           1       0.90      0.72      0.80        25

    accuracy                           0.83        52
   macro avg       0.84      0.82      0.82        52
weighted avg       0.84      0.83      0.82        52



### **Now use a dropout layer**

In [30]:
model = keras.Sequential([
                          keras.layers.Dense(60, input_dim = 60 , activation='relu'),
                          keras.layers.Dropout(0.5),
                          keras.layers.Dense(20,activation = 'relu'),
                          keras.layers.Dropout(0.5),
                          keras.layers.Dense(30,activation = 'relu'),
                          keras.layers.Dropout(0.5),
                          keras.layers.Dense(1,activation = 'sigmoid'),

])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

model.fit(X_train,y_train,epochs = 100, batch_size = 8)

Epoch 1/100
20/20 [==============================] - 1s 4ms/step - loss: 0.7752 - accuracy: 0.4615
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7059 - accuracy: 0.5128
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6862 - accuracy: 0.5321
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6983 - accuracy: 0.5128
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7278 - accuracy: 0.4679
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6955 - accuracy: 0.5641
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6847 - accuracy: 0.5513
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6886 - accuracy: 0.5513
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.7037 - accuracy: 0.5192
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6778 - accuracy: 0.5577
Epoch 11/

In [31]:
model.evaluate(X_test,y_test)

2/2 [==============================] - 0s 8ms/step - loss: 0.4289 - accuracy: 0.7885


[0.4288555979728699, 0.7884615659713745]

In [32]:
y_pred = model.predict(X_test).reshape(-1)
print(y_pred[:10])

y_pred = np.round(y_pred)
print(y_pred[:10])

[2.5487613e-04 6.9984514e-01 9.7414637e-01 2.8196538e-02 9.9982506e-01
 8.6021274e-01 5.1392221e-01 9.9992585e-01 1.9279568e-02 9.9997151e-01]
[0. 1. 1. 0. 1. 1. 1. 1. 0. 1.]


In [33]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.75      0.89      0.81        27
           1       0.85      0.68      0.76        25

    accuracy                           0.79        52
   macro avg       0.80      0.78      0.78        52
weighted avg       0.80      0.79      0.79        52

